<hr>

# Numerično integriranje

**Datum**: 10/11/2024

**Avtor**: Aleksander Grm

V zapiskih so uporabljeni primeri iz OnLine knjige [Numerične metode v ekosistemu Pythona, Janko Slavič](https://jankoslavic.github.io/pynm)

<hr>

Najprej naložimo celoten potreben Python ekosistem

In [ ]:
import numpy as np              # orodja za numeriko
import matplotlib.pyplot as mpl # izdelava grafov

# MatPlotLib set fonts
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['DejaVu Serif']

# MatPlotLib set LaTeX use
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{siunitx}'

import sympy as sym             # uporaba simbolične algebre
sym.init_printing()

from IPython.display import YouTubeVideo

<hr>

## Uvod

V okviru tega poglavja bomo za dano funkcijo $f(x)$ izračunali določen integral:

$$
    \int_a^b\,f(x)\,\textrm{d}x
$$

kjer sta $a$ in $b$ meji integriranja, $f(x)$ pa so vrednosti funkcije, ki jih pridobimo iz tabele vrednosti ali s pomočjo analitične funkcije.

Pri numeričnem integriranju integral ocenimo z $I$ in velja

$$
    \int_a^b\,f(x)\,\textrm{d}x= I + E,
$$

kjer je $E$ napaka ocene integrala. Numerični integral bomo računali na podlagi diskretne vsote:

$$
    I = \sum_{i=0}^{m}A_i\,f(x_i),
$$

kjer so $A_i$ uteži, $x_i$ pa vozlišča na intervalu $[a, b]$ in je $m+1$ število vozlišč.

Ogledali si bomo dva različna pristopa k numerični integraciji:

1. *Newton-Cotesov pristop*, ki temelji na ekvidistantnih vozliščih (konstanten korak integracije) in 
2. *Gaussov integracijski pristop*, kjer so vozlišča postavljena tako, da se doseže natančnost za polinome.

### Uvodni primer

Pri numeričnem integriranju si bomo pomagali s konkretnim primerom:

$$
    \int_1^2 x\,\sin(x)\,\textrm{d}x.
$$

Pripravimo si vozlišča. Osnovni korak naj bo $h=0.25$, v tem primeru imamo štiri podintervale in pet vozlišč, pri koraku $2h$ so tri vozliščne točke in pri koraku $4h$ samo dve (skrajni):

In [ ]:
xg, hg = np.linspace(1, 2, 100, retstep=True) # goste točke (za prikaz)
x2v, h2v = np.linspace(1, 2, 2, retstep=True)  # korak h2v = 1 (2 vozlišči)
x3v, h3v = np.linspace(1, 2, 3, retstep=True)  # korak h3v = 0.5 (3 vozlišča)
x4v, h4v = np.linspace(1, 2, 4, retstep=True)  # korak h4v = 0.33.. (4 vozlišča)
x5v, h5v = np.linspace(1, 2, 5, retstep=True)  # korak h5v = 0.25 (5 vozlišč)

Pripravimo še funkcijske vrednosti:

In [ ]:
yg = xg * np.sin(xg)
y2v = x2v * np.sin(x2v)
y3v = x3v * np.sin(x3v)
y4v = x4v * np.sin(x4v)
y5v = x5v * np.sin(x5v)

Pripravimo prikaz podatkov:

In [ ]:
def plot_intro_01():
    mpl.fill_between(xg, yg, alpha=0.25, facecolor='r')
    mpl.annotate(r'$\int_1^2\,x\,\sin(x)\,\textrm{d}x$', (1.3, 0.65), fontsize=22)
    mpl.plot(xg, yg, lw=3, alpha=0.5, label=r'$x\,\sin(x)$')
    mpl.plot(x2v, y2v, 's', alpha=0.5, label=f'$h={h2v}$', markersize=14)
    mpl.plot(x3v, y3v, 'o', alpha=0.5, label=f'$h={h3v}$', markersize=10)
    mpl.legend(loc=(0.05, 0.75))
    mpl.ylim(0, 2)
    mpl.show()

In [ ]:
plot_intro_01()

Integral lahko izračunamo tudi s pomočjo simbolične algebre

In [ ]:

sym.init_printing()
x = sym.symbols('x')
I_točno = float(sym.integrate(x*sym.sin(x), (x, 1, 2)).evalf())

print(I_točno)

<hr>

## Newton-Cotes kvadraturne formule - NC kvadrature

Prvi način numeričnega integriranja, ali kot pravimo **kvadraturne formule**, je z uporabo interpolacijskih polinomov in izpeljavo pravil. Na spodnji sliki je prikazan splošen primer delitve diskretnih točk funkcije $y_i = f(x_i)$, kjer je razdalja med vozlišči $x_i$ enaka $h$.

<center><img src="./figs/integriranje.png" alt="drawing" width="500"/></center>

Če je razdalja med delitvenimi točkami $x_i$ enaka, v našem primeru označimo z $h$, pravimo da so točke **ekvidistantno deljene**.

V okviru NC kvadratur si bomo pogledali tri metode:
- *Trapezno pravilo*
- *Simpsonova pravila*
- *Rombergovo metodo*

### Trapezno pravilo

Pri *trapeznem pravilu*, so vrednosti integralske funkcije $f(x)$ na intervalu $[x_0, x_1]$ interpolirane z linearno funkcijo. Za dve vozliščni točki to pomeni, da površino pod grafom funkcije približno izračunamo kot:

$$
    I_{\textrm{trapezno}}=\sum_{i=0}^{n=1}A_i\,f(x_i)=\frac{h}{2} \Big[ f(x_0)+f(x_1) \Big],
$$

kjer so uteži podane kot

$$
    A_0 = A_1 = \frac{1}{2}\,h.
$$

**Numerična implementacija**

In [ ]:
def trapezno(y, h):
    """
    Trapezno pravilo integriranja.
    
    :param y: funkcijske vrednosti
    :param h: korak integriranja
    """
    return (y[0] + y[-1])*h/2

**Primer**

Uporabimo vrednosti iz uvodnega primera

In [ ]:
I_trapezno = trapezno(y2v, h=h2v)
I_trapezno

In [ ]:
def plot_trapezno():
    mpl.fill_between(x2v, y2v, alpha=0.25, facecolor='r')
    mpl.vlines(x2v, 0, y2v, color='r', linestyles='dashed', lw=1)
    mpl.annotate('$I_{\\textrm{trapezno}}$', (1.4, 0.5), fontsize=22)
    mpl.annotate('Napaka', fontsize=20, xy=(1.5, 1.4), xytext=(1.6, 1.8),
            arrowprops=dict(facecolor='gray', shrink=0.05))
    mpl.plot(xg, yg, lw=3, alpha=0.5, label='$x\,\sin(x)$')
    mpl.plot(x2v, y2v, 'o', alpha=0.5, label=f'$h={h2v}$')
    mpl.legend(loc=(1.01, 0))
    mpl.ylim(0, 2)
    mpl.show()

In [ ]:
plot_trapezno()

### Napaka trapeznega pravila

Razlika med analitično vrednostjo integrala in numeričnim približkom $I$ je napaka metode:

$$
    E = \int_a^b f(x)\,dx - I.
$$

Če je funkcija $f(x)$ vsaj dvakrat odvedljiva, lahko izpeljemo oceno za napako, ki velja samo za trapezni približek prek celega integracijskega intervala:

$$
    E_{\textrm{trapezno}} = -\frac{h^3}{12} f''(\xi),
$$

kjer je $h=b-a$ in $\xi$ neznana vrednost na intervalu $[a,b]$.

### Sestavljeno trapezno pravilo

Če razdelimo interval $[a, b]$ na $n$ podintervalov in na vsakem uporabimo trapezno pravilo integriranja,  govorimo o *sestavljenem trapeznem pravilu* (angl. *composite trapezoidal rule*).

V tem primeru za vsak podinterval $i$ uporabimo trapezno pravilo in torej za meje podinterval $x_i$ in $x_{i+1}$ uporabimo uteži $A_i=A_{i+i}=h/2$. Ker so notranja vozlišča podvojena, sledi:

$$
    A_0 = A_{n} = \frac{h}{2}\quad\textrm{in za ostala vozlišča:}\quad A_i=h.
$$

Pri tem smo imeli v mislih intervale enake širine:

$$
    h=\frac{x_{n}-x_0}{n}.
$$

Tako sledi

$$
    I_{\textrm{trapezno sest}} = \sum_{i=0}^{n} A_i\,f(x_i) = \left(\frac{y_0}{2} + y_1+y_2+\cdots+y_{n-1}+\frac{y_{n}}{2}\right)\,h.
$$

**Numerična implementacija**

In [ ]:
def trapezno_sest(y, h):
    """
    Sestavljeno trapezno pravilo integriranja.
    
    :param y: funkcijske vrednosti
    :param h: korak integriranja
    """    
    return (np.sum(y) - 0.5*y[0] - 0.5*y[-1])*h

**Primer**

Uporabimo uvodni primer, kjer vzamemo tri točkovno delitev intervala

In [ ]:
print('točke:', x3v, ', delitev h:', h3v)

Z uporabo implementirane metode lahko določimo površino pod krivuljo

In [ ]:
I_trapezno_sest = trapezno_sest(y3v, h=h3v)
I_trapezno_sest

In [ ]:
def plot_trapezno_sest():
    mpl.fill_between(x3v, y3v, alpha=0.25, facecolor='r')
    mpl.vlines(x3v, 0, y3v, color='r', linestyles='dashed', lw=1)
    mpl.annotate('$I_{\\textrm{trapezno sestavljeno}}$', (1.2, 0.5), fontsize=22)
    mpl.annotate('Napaka', fontsize=20, xy=(1.75, 1.68), xytext=(1.4, 1.8),
            arrowprops=dict(facecolor='gray', shrink=0.05))
    mpl.annotate('Napaka', fontsize=20, xy=(1.2, 1.1), xytext=(1., 1.4),
            arrowprops=dict(facecolor='gray', shrink=0.05))
    mpl.plot(xg, yg, lw=3, alpha=0.5, label='$x\,\sin(x)$')
    mpl.plot(x3v, y3v, 'o', alpha=0.5, label=f'$h={h3v}$')
    mpl.legend(loc=(1.01, 0))
    mpl.ylim(0, 2)
    mpl.show()

In [ ]:
plot_trapezno_sest()

### Napaka sestaljenega trapeznega pravila

Napaka sestavljenega trapeznega pravila izhaja iz napake trapeznega pravila; pri tem tako napako naredimo $n$-krat.

Ker velja $h\cdot n=b-a$, izpeljemo napako sestavljenega trapeznega pravila kot:

$$
    E_{\textrm{trapezno sest}} = -\frac{h^2(b-a)}{12} f''(\eta),
$$

$\eta$ je vrednost na intervalu $[a,b]$. Napaka je drugega reda $\mathcal{O}(h^2)$.

### `numpy` implementacija sestavljenega trapeznega pravila

Sestavljeno trapezno pravilo je implementirano tudi v paketu `numpy`, s funkcijo `numpy.trapezoid`:

```python
trapezoid(y, x=None, dx=1.0, axis=-1)
```

* `y` predstavlja tabelo funkcijskih vrednosti, 
* `x` je opcijski parameter in definira vozlišča; če parameter ni definiran, se privzame ekvidistančna vozlišča na razdalji `dx`,
* `dx` definira (konstanten) korak integracije, ima privzeto vrednost 1,
* `axis` definira *os* po kateri se integrira (v primeru, da je `y` večdimenzijsko numerično polje).

Funkcija vrne izračunani integral po sestavljenem trapeznem pravilu. Več informacij lahko najdete v [dokumentaciji](https://numpy.org/doc/stable/reference/generated/numpy.trapezoid.html).

**Primer uporabe**

In [ ]:
I_trapezno_np = np.trapz(y3v, dx=h3v)
I_trapezno_np

<hr>

## Simpsonova pravila

Zgoraj smo si pogledali trapezno pravilo, ki temelji na linearni interpolacijski funkciji na posameznem podintervalu. Z interpolacijo višjega reda lahko izpeljemo še druge integracijske metode.

Problem je izračunati integral, kjer naj bo natančnost višja

$$
    \int_{a}^b f(x)\,dx.
$$

Tabeliramo podintegralsko funkcijo $f(x)$ in tabelo interpoliramo z Lagrangevim interpolacijskim polinomom $P_n(x)$ stopnje $n$:

$$
    P_n(x) = \sum_{i=0}^{n}\,f(x_i)\,l_i(x),
$$

kjer so $y_i=f(x_i)$ funkcijske vrednosti v vozliščih $x_i$ in je Lagrangev polinom $l_i$ definiran kot (poglej si pod poglavje *2. Interpolacija*, kaj smo počeli):

$$
    l_i(x)=\prod_{j=0, j\ne i}^n \frac{x-x_j}{x_i-x_j}.
$$

Za numerični izračun integrala $\int_{a}^b f(x)\,dx$ (meje so: $a=x_0$, $b=x_n$) namesto funkcije $f(x)$ vstavimo v integral interpolacijski polinom $P_n(x)$:

$$
    I = \int_{x_0}^{x_{n}} P_n(x)\,dx = \int_{x_0}^{x_{n}} \sum_{i=0}^{n}\,f(x_i)\,l_i(x)\,dx.
$$

Ker je integriranje linearna operacija, lahko zamenjamo integriranje in vsoto:

$$
    I = \sum_{i=0}^{n}\,f(x_i)\,\underbrace{\int_{x_0}^{x_{n}} l_i(x)\,dx}_{A_i}.
$$

Lagrangev polinom integriramo in dobimo uteži $A_i$:

$$
    A_i = \int_{x_0}^{x_{n}} l_i(x)\,dx.
$$

### Izpeljava `Trapeznega pravila` s pomočjo lagrangejevega polinoma

Poglejmo si kako z Lagrangevim interpolacijskim polinomom prve stopnje strojno izpeljemo uteži $A_i$ v primeru trapeznega pravila.

Najprej v simbolni obliki definirajmo spremenljivko `x`, vozlišči `x0` in `x1` ter korak `h`:

In [ ]:
x, x0, x1, h = sym.symbols('x x0, x1, h')

Pripravimo Python funkcijo, ki v simbolni obliki vrne seznam $n$ koeficientov Lagrangevih polinomov $[l_0(x), l_1(x),\dots, l_{n-1}(x)]$ stopnje $n-1$:

In [ ]:
def lagrange(n, x, vozlišča_predpona='x'):
    if isinstance(vozlišča_predpona, str):
        vozlišča = sym.symbols(f'{vozlišča_predpona}:{n}') 
    coeffs = []
    for i in range(0, n):
        numer = []
        denom = []

        for j in range(0, n):
            if i == j:
                continue

            numer.append(x    - vozlišča[j])
            denom.append(vozlišča[i] - vozlišča[j])
        
        numer = sym.Mul(*numer)
        denom = sym.Mul(*denom)

        coeffs.append(numer/denom)
    return coeffs    

Najprej poglejmo Lagrangeva polinoma za linearno interpolacijo ($n=2$):

In [ ]:
lag = lagrange(2, x)
lag

Sedaj Lagrangev polinom $l_0(x)$ integriramo čez celotni interval:

In [ ]:
int0 = sym.integrate(lag[0], (x, x0, x1))
int0

Izraz poenostavimo in dobimo:

In [ ]:
int1 = int0.factor()
int1

Ker je širina podintervala konstantna je $x_1=h_0+h$, izvedemo zamenjavo:

In [ ]:
zamenjave = {x1: x0+h}
int1.subs(zamenjave)

Zgornje korake za Lagrangev polinom  $l_0(x)$ lahko posplošimo za seznam Lagrangevih polinomov:

In [ ]:
x, x0, x1, h = sym.symbols('x, x0, x1, h')
zamenjave = {x1: x0+h}
A_trapez = [sym.integrate(li, (x, x0, x1)).factor().subs(zamenjave) 
            for li in lagrange(2, x)]  # za vsak lagrangev polimom `li` v seznamu lagrange(2,x)
A_trapez

Izpeljali smo uteži, ki smo jih uporabili pri trapezni metodi:

$$
    A_0 = h/2\qquad A_1=h/2.
$$

Trapezno pravilo je:

$$
    I_{\textrm{trapezno}} = \frac{h}{2}\left(y_0+y_1\right).
$$

Ocena napake je:

$$
    E_{\textrm{trapezno}} = -\frac{h^3}{12}f''(\xi).
$$

### Uteži pri Simpsonovem pravilu

Na identičen način, kakor smo izpeljali trapezno pravilo, lahko naredimo enako za **kvadratno interpolacijsko funkcijo**, $y = a_0 + a_1 x + a_2 x^2$, kjer uporabimo tri točke ($n=3$).

Izračun uteži je analogen zgornjemu:

In [ ]:
x, x0, x1, x2, h = sym.symbols('x, x0, x1, x2, h')
zamenjave = {x1: x0+h, x2: x0+2*h}
A_Simpson1_3 = [sym.integrate(li, (x, x0, x2)).factor().subs(zamenjave).factor() 
             for li in lagrange(3, x)]
A_Simpson1_3

Simpsonovo pravilo (to pravilo se imenuje tudi Simpsonovo 1/3 pravilo) je:

$$
    I_{\textrm{Simpsonovo}} = \frac{h}{3}\left(y_0+4\,y_1+y_2\right).
$$

Ocena za napako je podana z:

$$
    E_{\textrm{Simpsonovo}} = -\frac{h^5}{90}f^{(4)}(\xi).
$$

Pri tem je treba izpostaviti, da je napaka lokalno 5 reda  $\mathcal{O}(h^5)$ in definirana z neznano vrednostjo četrtega odvoda $f^{(4)}(\xi)$, posledično je to pravilo točno za polinome stopnje 3 ali manj.

**Primer**

Uporabimo uvodni primer za primerjavo

In [ ]:
I_Simps = h3v/3 * np.sum(y3v * [1, 4, 1])
I_Simps

Pripravimo sliko. Ker Simsonovo pravilo temelji na kvadratni interpolaciji, moramo naprej pripraviti interpolacijski polinom (pomagamo si s ``scipy.interpolate`` - [dokumentacija](https://docs.scipy.org/doc/scipy-1.14.1/tutorial/interpolate.html))

In [ ]:
from scipy import interpolate

In [ ]:
def plot_Simpson():
    y_interpolate = interpolate.lagrange(x3v, y3v)
    mpl.fill_between(xg, y_interpolate(xg), alpha=0.25, facecolor='r')
    mpl.vlines(x3v, 0, y3v, color='r', linestyles='dashed', lw=1)
    mpl.annotate('$I_{\\textrm{Simpsonovo}}$', (1.2, 0.5), fontsize=22)
    mpl.annotate('Napaka', fontsize=20, xy=(1.75, 1.7), xytext=(1.4, 1.8),
            arrowprops=dict(facecolor='gray', shrink=0.05))
    mpl.annotate('Napaka', fontsize=20, xy=(1.2, 1.1), xytext=(1., 1.4),
            arrowprops=dict(facecolor='gray', shrink=0.05))
    mpl.plot(xg, yg, lw=3, alpha=0.5, label='$x\,\sin(x)$')
    mpl.plot(x3v, y3v, 'o', alpha=0.5, label=f'$h={h3v}$')
    mpl.legend(loc=(1.01, 0))
    mpl.ylim(0, 2)
    mpl.show()

In [ ]:
plot_Simpson()

### Uporaba `scipy.integrate.newton_cotes` paketa

Koeficiente integracijskega pristopa Newton-Cotes pridobimo tudi s pomočjo `scipy.integrate.newton_cotes()` [dokumentacija](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.newton_cotes.html):

```python
newton_cotes(rn, equal=0)
```

kjer sta parametra:

* `rn`, ki definira število podintervalov (mogoč je tudi nekonstanten korak, glejte [dokumentacijo](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.newton_cotes.html)),
* `equal`, ki definira ali se zahteva konstantno široke podintervale.

Funkcija vrne seznam, pri čemer prvi element predstavlja numerično polje uteži in drugi člen oceno napake.

**Primer**

Poglejmo si primer uporabe

In [ ]:
from scipy import integrate

res = integrate.newton_cotes(2)
print('uteži:', res[0])
print('napaka:', res[1])

### Simpson 3/8 pravilo

Nadaljujemo lahko s kubično interpolacijo čez štiri točke ($n=4$):

In [ ]:
x, x0, x1, x2, x3, h = sym.symbols('x, x0, x1, x2, x3, h')
zamenjave = {x1: x0+h, x2: x0+2*h, x3: x0+3*h}
A_Simpson3_8 = [sym.integrate(li, (x, x0, x3)).factor().subs(zamenjave).factor() 
                for li in lagrange(4, x)]
A_Simpson3_8

Simpsonovo 3/8 pravilo je:

$$
    I_{\textrm{Simpsonovo 3/8}} = \frac{3h}{8}\left(y_0+3\,y_1+3\,y_2+y_3\right).
$$

Ocena za napako:

$$
    E_{\textrm{Simpsonovo 3/8}} = -\frac{3\,h^5}{80}f^{(4)}(\xi).
$$

**Primer**

Uporabimo pripravljeno tabelo vrednosti funkcije v štirih točkah iz uvodnega primera

In [ ]:
print('točke:', y4v)
print('korak:', h4v)

Izračunamo z uporabo Simpson 3/8

In [ ]:
I_Simps38 = 3*h4v/8 * np.sum(y4v * [1, 3, 3, 1])
I_Simps38

In [ ]:
def plot_Simpson38():
    y_interpolate = interpolate.lagrange(x4v, y4v)
    mpl.fill_between(xg, y_interpolate(xg), alpha=0.25, facecolor='r')
    mpl.vlines(x4v, 0, y4v, color='r', linestyles='dashed', lw=1)
    mpl.annotate('$I_{\\textrm{Simpsonovo 3/8}}$', (1.2, 0.5), fontsize=22)
    mpl.annotate('Napaka', fontsize=20, xy=(1.75, 1.7), xytext=(1.4, 1.8),
            arrowprops=dict(facecolor='gray', shrink=0.05))
    mpl.annotate('Napaka', fontsize=20, xy=(1.5, 1.47), xytext=(1.1, 1.6),
            arrowprops=dict(facecolor='gray', shrink=0.05))
    mpl.annotate('Napaka', fontsize=20, xy=(1.2, 1.1), xytext=(1., 1.4),
            arrowprops=dict(facecolor='gray', shrink=0.05))    
    mpl.plot(xg, yg, lw=3, alpha=0.5, label='$x\,\sin(x)$')
    mpl.plot(x4v, y4v, 'o', alpha=0.5, label=f'$h={h4v:.6f}$')
    mpl.legend(loc=(1.01, 0))
    mpl.ylim(0, 2)
    mpl.show()

In [ ]:
plot_Simpson38()

### Sestavljeno Simpsonovo pravilo

Interval $[a, b]$ razdelimo na sodo število $n$ podintervalov enake širine $h=(b - a)/n$, s čimer so definirana vozlišča $x_i=a+i\,h$ za $i=0,1,\dots,n$. V tem primeru zapišemo sestavljeno Simpsonovo pravilo:

$$
    \int_{a}^{b}f(x)\,\textrm{d}x =
    \frac{h}{3}\left(
    f(x_0)
    +4\sum_{i=1}^{n/2}f(x_{2i-1})
    +2\sum_{i=1}^{n/2-1}f(x_{2i})
    +f(x_n)
    \right)
    \underbrace{
    -\frac{b-a}{180}h^4\,f^{(4)}(\eta)
    }_{E_{\textrm{Sestavljeno Simpsonovo 1/3}}},
$$

kjer je $\eta$ neznana vrednost na intervalu $[a, b]$. Napaka je četrtega reda $\mathcal{O}(h^4)$.

**Numerična implementacija**

In [ ]:
def simpsonovo_sest(y, h):
    """
    Sestavljeno Simpsonovo pravilo integriranja.

    :param y: funkcijske vrednosti
    :param h: korak integriranja
    """    
    return h/3 * (y[0] + 4*np.sum(y[1:-1:2]) + 2*np.sum(y[2:-1:2]) + y[-1])

**Primer**

Uporabimo delitev iz uvodnega primera

In [ ]:
I_Simps_sest = simpsonovo_sest(y5v, h=h5v)
I_Simps_sest

In [ ]:
def plot_Simpson_sest():
    y_interpolate = interpolate.lagrange(x5v, y5v)
    mpl.fill_between(xg, y_interpolate(xg), alpha=0.25, facecolor='r')       
    mpl.vlines(x5v, 0, y5v, color='r', linestyles='dashed', lw=1)
    mpl.annotate('$I_{\\textrm{Simpsonovo sestavljeno}}$', (1.2, 0.5), fontsize=22)
    mpl.annotate('Napaka', fontsize=20, xy=(1.70, 1.68), xytext=(1.4, 1.8),
            arrowprops=dict(facecolor='gray', shrink=0.05))
    mpl.plot(xg, yg, lw=3, alpha=0.5, label='$x\,\sin(x)$')
    mpl.plot(x5v, y5v, 'o', alpha=0.5, label=f'$h={h5v}$')
    mpl.legend(loc=(1.01, 0))
    mpl.ylim(0, 2)
    mpl.show()

In [ ]:
plot_Simpson_sest()

## Gaussove kvadraturne formule - Gaussove kvadrature

Newton-Cotesov pristop temelji na polinomu $n$-te stopnje in napaka je $n+1$ stopnje. To pomeni, da integracija daje točen rezultat, če je integrirana funkcija polinom stopnje $n$ ali manj.

Gaussov integracijski pristop je v principu drugačen: cilj je integral funkcije $f(x)$ nadomestiti z uteženo vsoto vrednosti funkcije pri diskretnih vrednostih $f(x_i)$:

$$
    \int_a^bf(x)\,\textrm{d}x\approx \sum_{i=0}^{n-1} A_i\, f(x_i).
$$

Pri tem je utež $A_i$ **neznana** in tudi lega vozlišča $x_i$ je **neznana**. Za stopnje polinoma $n$ bomo potrebovali tudi več  točk $(x_i, f(x_i))$.

V nadaljevanju bomo spoznali, da lahko zelo učinkovito in natančno numerično izračunamo  integral. Prednost Gaussove integracije je tudi, da lahko izračuna neprave integrale (npr: $\int_0^1\sin(x)/\sqrt{(x)}\,dx$ v točki $x=0$ integral divergira).

Predpostavimo, da želimo integrirati polinom stopnje $n=1$ (linearna funkcija):

$$
    f(x) = P_1(x) = a_0+a_1\,x.
$$

Izračunajmo: 

$$
    \int_{x_L}^{x_D}P_1(x)\,\textrm{d}x = \left(a_0\,x+a_1\,\frac{x^2}{2}\right)_{x_L}^{x_D} = -a_0\,x_L+a_0\,x_D-\frac{a_1\,x_L^2}{2}+\frac{a_1\,x_D^2}{2}.
$$

Po drugi strani pa želimo integral izračunati glede na ustrezno uteženo $A_0$ vrednost funkcije $f(x_0)$ v neznanem vozlišču $x_0$ (samo eno vozlišče!):

$$
    \int_{x_L}^{x_D}P_1(x)\,\textrm{d}x = A_0\,P_1(x_0)= A_0\,a_0+A_0\,a_1\,x_0.
$$

Z enačenjem zgornjih izrazov izpeljemo:

$$-a_0\,x_L+a_0\,x_D-\frac{a_1\,x_L^2}{2}+\frac{a_1\,x_D^2}{2}=A_0\,a_0+A_0\,a_1\,x_0.$$

$a_0$ in $a_1$ sta koeficienta linearne funkcije, ki lahko zavzame poljubne vrednosti, zato velja:

$$a_0\,\left(-x_L+x_D-A_0\right)=0\qquad\textrm{in}\qquad a_1\left(-\frac{x_L^2}{2}+\frac{x_D^2}{2}-A_0\,x_0\right)=0.$$

Gre za sistem linearnih enačb z rešitvijo:

$$A_0= x_D-x_L, \qquad x_0=\frac{x_L+x_D}{2}.$$

Če je integrirana funkcija linearna, bomo samo na podlagi vrednosti v eni točki izračunali pravo vrednost!

Da je Gaussov integracijski pristop neodvisen od mej integriranja $x_L$, $x_D$, pa uvedemo standardne meje.

#### Standardne meje: $x_L=-1$ in $x_D=1$

Zaradi splošnosti meje $x\in[x_L, x_D]$ transformiramo v $\xi\in[-1, +1]$ s pomočjo:

$$x=\frac{x_D+x_L}{2}+\frac{x_D-x_L}{2}\xi$$

in

$$\textrm{d}x=\frac{x_D-x_L}{2}\textrm{d}\xi.$$

Velja:

$$\int_{x_L}^{x_D}f(x)\,\textrm{d}x=\int_{-1}^1 g\left(\xi\right)\,\textrm{d}\xi,$$

kjer je:

$$g(\xi)=\frac{x_D-x_L}{2}\,f\left(\frac{x_L+x_d}{2}+\frac{x_D-x_L}{2}\xi\right).$$

V primeru standardnih mej, je pri eni Gaussovi točki utež $A_0=2$ in $x_0=0$ vrednost, pri kateri moramo izračunati funkcijo $f(x_0)$.

#### Strojno izpeljevanje uteži in Gaussove točke

Definirajmo simbole in nastavimo enačbo:

In [ ]:
a_0, a_1, x, x_L, x_D, A_0, x_0 = sym.symbols('a_0, a_1, x, x_L, x_D, A_0, x_0') # simboli
P1 = a_0 + a_1*x                                                   # linearni polinom
eq = sym.Eq(P1.integrate((x, x_L, x_D)).expand(), A_0*P1.subs(x, x_0)) # teoretični integral = ocen z utežmi
eq

Pripravimo dve enačbi (za prvo predpostavimo $a_0=0, a_1=1$, za drugo predpostavimo $a_0=1, a_1=0$) ter rešimo sistem za `A_0` in `x_0`:

In [ ]:
sym.solve([eq.subs(a_0, 0).subs(a_1, 1), eq.subs(a_1, 0).subs(a_0, 1)], [A_0, x_0])

Izpeljavo z več vozlišči poteka podobno. Kot vidimo, je potrebno rešiti sistem enačb, kjer so rešitve delitvene točke $x_i$ in uteži $A_i$.